# Import the necessary packages required to extract tweets


In [1]:

import logging
import queue
import threading
import time
import argparse

import tweepy
import dropbox

# Create a twitter developer account 

# Create a dropbox account

#Used twitter consumer key,token 
#acces key, token
#Dropbox access token

In [2]:
TWITTER_CONSUMER_KEY = ''  # to fill
TWITTER_CONSUMER_SECRET = ''  # to fill
TWITTER_ACCESS_TOKEN = ''  # to fill
TWITTER_ACCESS_SECRET = ''  # to fill

DROPBOX_ACCESS_TOKEN = ''  # to fill
DEFAULT_SAVE_SIZE = 4 * 1024 ** 2  # 4 MB

LIMIT_SLEEP = 10

LOCATION_CHICAGO = [-87.94011, 41.64454, -87.52413, 42.02303]

LOGGING_FORMAT = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.INFO, format=LOGGING_FORMAT)


class JSONDropboxSaver(threading.Thread):

    def __init__(self, queue, access_token, save_size):
        super().__init__()
        self.queue = queue

        self.access_token = access_token
        self.dbx = dropbox.Dropbox(self.access_token)

        self.save_size = save_size

    def run(self):
        data = []
        accumulated_data_length = 0
        try:
            while True:
                item = self.queue.get()
                if item is None:
                    logging.info('Got None in Queue, saving and ending thread...')
                    if data:
                        self._save_json(data)
                    break
                else:
                    data.append(item)
                    accumulated_data_length += len(item)
                    logging.info('Accumulated Data Length: {}'.format(accumulated_data_length))
                if accumulated_data_length > self.save_size:
                    self._save_json(data)
                    data = []
                    accumulated_data_length = 0

        except Exception:
            logging.exception('Exception {}!'.format(e))

    def _save_json(self, data):
        json_data = '[' + ','.join(data) + ']'
        path = '/' + time.strftime('%Y-%m-%d %H:%M:%S') + '.json'
        logging.info('Saving to file <{}>. {} bytes...'.format(path, len(json_data)))
        self.dbx.files_upload(json_data.encode('utf8'), path)


class TweetsQueueListner(tweepy.streaming.StreamListener):
    def __init__(self, queue):
        self.queue = queue

    def on_connect(self):
        logging.info('StreamListner connected succefully to Twitter!')

    def on_data(self, data):
        logging.info('Data arrived! len={}'.format(len(data)))
        if data:
            self.queue.put(data)

    def on_error(self, status_code):
        logging.error('Stream Error! status code: {}'.format(status_code))
        if status_code != 420:
            return False

    # TODO - better sleep mechanisem
    def on_limit(self, track):
        logging.warning('Limit reached! <{}>'.format(str(track)))
        logging.warning('Sleeping for {} seconds...'.LIMIT_SLEEP)
        time.sleep(LIMIT_SLEEP)
        logging.warning('Woke up!')


def main(save_size):
    logging.info('Authenticating...')
    auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
    auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_SECRET)
    logging.info('Authentication succeed!')

    tweets_queue = queue.Queue()

    tweets_listener = TweetsQueueListner(tweets_queue)

    dropbox_saver = JSONDropboxSaver(tweets_queue, DROPBOX_ACCESS_TOKEN, save_size)
    logging.info('Starting Dropbox Save thread...')
    dropbox_saver.start()

    tweets_stream = tweepy.Stream(auth, tweets_listener)

    logging.info('Filtering Tweets by location in {}...'.format(LOCATION_CHICAGO))
    tweets_stream.filter(locations=LOCATION_CHICAGO, is_async=False)

    try:
        pass  # dropbox_saver.join()

    # BaseException for catching also KeyboardInterrupt
    except BaseException as e:
        logging.exception('Exception {}!'.format(e))

        logging.info('Disconnect Tweets Stream...')
        tweets_stream.disconnect()

        logging.info('Put None in Queue...')
        tweets_queue.put(None)

        logging.info('Wait Dropbox Saver to finish...')
        dropbox_saver.join()

    finally:
        logging.info('Bye Bye...')


if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='Tweets saver to Dropbox.')

    parser.add_argument('-s', '--save-size', type=int, default=DEFAULT_SAVE_SIZE,
                        help='save size in bytes')

    parser.add_argument('-l', '--log', type=str,
                        help='log file path')
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
    args = parser.parse_args()

    if args.log:
        log_file = logging.FileHandler(args.log)
        log_file.setLevel(logging.DEBUG)
        formatter = logging.Formatter(LOGGING_FORMAT)
        log_file.setFormatter(formatter)
        logging.getLogger('').addHandler(log_file)

    main(args.save_size)

2019-07-31 13:12:54,851 Authenticating...
2019-07-31 13:12:54,853 Authentication succeed!
2019-07-31 13:12:54,854 Starting Dropbox Save thread...
2019-07-31 13:12:54,857 Filtering Tweets by location in [-87.94011, 41.64454, -87.52413, 42.02303]...
2019-07-31 13:12:55,046 StreamListner connected succefully to Twitter!
2019-07-31 13:12:57,348 Data arrived! len=2930
2019-07-31 13:12:57,349 Accumulated Data Length: 2930
2019-07-31 13:12:58,270 Data arrived! len=4048
2019-07-31 13:12:58,272 Accumulated Data Length: 6978
2019-07-31 13:12:58,469 Data arrived! len=7508
2019-07-31 13:12:58,471 Accumulated Data Length: 14486
2019-07-31 13:13:00,113 Data arrived! len=2700
2019-07-31 13:13:00,114 Accumulated Data Length: 17186
2019-07-31 13:13:04,415 Data arrived! len=5095
2019-07-31 13:13:04,417 Accumulated Data Length: 22281
2019-07-31 13:13:09,369 Data arrived! len=7522
2019-07-31 13:13:09,371 Accumulated Data Length: 29803
2019-07-31 13:13:10,660 Data arrived! len=4008
2019-07-31 13:13:10,662 

2019-07-31 13:15:27,983 Data arrived! len=5596
2019-07-31 13:15:27,984 Accumulated Data Length: 330615
2019-07-31 13:15:31,977 Data arrived! len=2558
2019-07-31 13:15:31,979 Accumulated Data Length: 333173
2019-07-31 13:15:34,127 Data arrived! len=2578
2019-07-31 13:15:34,128 Accumulated Data Length: 335751
2019-07-31 13:15:34,293 Data arrived! len=3744
2019-07-31 13:15:34,295 Accumulated Data Length: 339495
2019-07-31 13:15:34,639 Data arrived! len=3214
2019-07-31 13:15:34,640 Accumulated Data Length: 342709
2019-07-31 13:15:35,868 Data arrived! len=7395
2019-07-31 13:15:35,869 Accumulated Data Length: 350104
2019-07-31 13:15:36,281 Data arrived! len=2609
2019-07-31 13:15:36,283 Accumulated Data Length: 352713
2019-07-31 13:15:38,735 Data arrived! len=3915
2019-07-31 13:15:38,737 Accumulated Data Length: 356628
2019-07-31 13:15:39,145 Data arrived! len=6268
2019-07-31 13:15:39,147 Accumulated Data Length: 362896
2019-07-31 13:15:39,165 Data arrived! len=8924
2019-07-31 13:15:39,165 Ac

2019-07-31 13:17:48,479 Data arrived! len=4791
2019-07-31 13:17:48,480 Accumulated Data Length: 649723
2019-07-31 13:17:50,014 Data arrived! len=4169
2019-07-31 13:17:50,016 Accumulated Data Length: 653892
2019-07-31 13:17:54,009 Data arrived! len=5344
2019-07-31 13:17:54,010 Accumulated Data Length: 659236
2019-07-31 13:18:05,375 Data arrived! len=2693
2019-07-31 13:18:05,376 Accumulated Data Length: 661929
2019-07-31 13:18:06,604 Data arrived! len=2737
2019-07-31 13:18:06,605 Accumulated Data Length: 664666
2019-07-31 13:18:07,730 Data arrived! len=2749
2019-07-31 13:18:07,732 Accumulated Data Length: 667415
2019-07-31 13:18:10,290 Data arrived! len=2875
2019-07-31 13:18:10,292 Accumulated Data Length: 670290
2019-07-31 13:18:15,105 Data arrived! len=2559
2019-07-31 13:18:15,107 Accumulated Data Length: 672849
2019-07-31 13:18:17,868 Data arrived! len=2578
2019-07-31 13:18:17,869 Accumulated Data Length: 675427
2019-07-31 13:18:20,530 Data arrived! len=2675
2019-07-31 13:18:20,531 Ac

2019-07-31 13:21:09,392 Data arrived! len=3429
2019-07-31 13:21:09,393 Accumulated Data Length: 965133
2019-07-31 13:21:10,007 Data arrived! len=2754
2019-07-31 13:21:10,008 Accumulated Data Length: 967887
2019-07-31 13:21:11,134 Data arrived! len=2526
2019-07-31 13:21:11,135 Accumulated Data Length: 970413
2019-07-31 13:21:11,850 Data arrived! len=2819
2019-07-31 13:21:11,851 Accumulated Data Length: 973232
2019-07-31 13:21:15,331 Data arrived! len=2792
2019-07-31 13:21:15,332 Accumulated Data Length: 976024
2019-07-31 13:21:16,867 Data arrived! len=6924
2019-07-31 13:21:16,868 Accumulated Data Length: 982948
2019-07-31 13:21:18,199 Data arrived! len=3409
2019-07-31 13:21:18,200 Accumulated Data Length: 986357
2019-07-31 13:21:23,319 Data arrived! len=6231
2019-07-31 13:21:23,320 Accumulated Data Length: 992588
2019-07-31 13:21:23,467 Data arrived! len=2995
2019-07-31 13:21:23,468 Accumulated Data Length: 995583
2019-07-31 13:21:24,241 Data arrived! len=6849
2019-07-31 13:21:24,242 Ac

2019-07-31 13:23:43,508 Data arrived! len=5239
2019-07-31 13:23:43,509 Accumulated Data Length: 1273146
2019-07-31 13:23:44,328 Data arrived! len=5645
2019-07-31 13:23:44,329 Accumulated Data Length: 1278791
2019-07-31 13:23:46,375 Data arrived! len=4264
2019-07-31 13:23:46,376 Accumulated Data Length: 1283055
2019-07-31 13:23:47,297 Data arrived! len=3449
2019-07-31 13:23:47,298 Accumulated Data Length: 1286504
2019-07-31 13:23:48,116 Data arrived! len=5073
2019-07-31 13:23:48,117 Accumulated Data Length: 1291577
2019-07-31 13:23:49,550 Data arrived! len=2848
2019-07-31 13:23:49,551 Accumulated Data Length: 1294425
2019-07-31 13:23:53,645 Data arrived! len=4990
2019-07-31 13:23:53,647 Accumulated Data Length: 1299415
2019-07-31 13:23:56,205 Data arrived! len=4270
2019-07-31 13:23:56,207 Accumulated Data Length: 1303685
2019-07-31 13:23:56,615 Data arrived! len=6434
2019-07-31 13:23:56,616 Accumulated Data Length: 1310119
2019-07-31 13:23:57,332 Data arrived! len=2852
2019-07-31 13:23:

...

2019-07-31 16:28:40,153 Accumulated Data Length: 1707546
2019-07-31 16:28:40,766 Data arrived! len=2756
2019-07-31 16:28:40,768 Accumulated Data Length: 1710302
2019-07-31 16:28:49,675 Data arrived! len=2648
2019-07-31 16:28:49,676 Accumulated Data Length: 1712950
2019-07-31 16:28:50,187 Data arrived! len=9645
2019-07-31 16:28:50,189 Accumulated Data Length: 1722595
2019-07-31 16:28:54,078 Data arrived! len=4893
2019-07-31 16:28:54,080 Accumulated Data Length: 1727488
2019-07-31 16:28:54,081 Data arrived! len=6635
2019-07-31 16:28:54,084 Accumulated Data Length: 1734123
2019-07-31 16:28:55,921 Data arrived! len=2482
2019-07-31 16:28:55,923 Accumulated Data Length: 1736605
2019-07-31 16:29:00,121 Data arrived! len=6660
2019-07-31 16:29:00,122 Accumulated Data Length: 1743265
2019-07-31 16:29:03,355 Data arrived! len=3262
2019-07-31 16:29:03,356 Accumulated Data Length: 1746527
2019-07-31 16:29:03,704 Data arrived! len=3400
2019-07-31 16:29:03,706 Accumulated Data Length: 1749927
2019-07

2019-07-31 16:30:53,174 Accumulated Data Length: 2013451
2019-07-31 16:30:55,835 Data arrived! len=2652
2019-07-31 16:30:55,837 Accumulated Data Length: 2016103
2019-07-31 16:30:57,576 Data arrived! len=6603
2019-07-31 16:30:57,578 Accumulated Data Length: 2022706
2019-07-31 16:30:59,318 Data arrived! len=2841
2019-07-31 16:30:59,319 Accumulated Data Length: 2025547
2019-07-31 16:31:00,033 Data arrived! len=2667
2019-07-31 16:31:00,035 Accumulated Data Length: 2028214
2019-07-31 16:31:02,185 Data arrived! len=2833
2019-07-31 16:31:02,186 Accumulated Data Length: 2031047
2019-07-31 16:31:02,901 Data arrived! len=6051
2019-07-31 16:31:02,903 Accumulated Data Length: 2037098
2019-07-31 16:31:03,926 Data arrived! len=2782
2019-07-31 16:31:03,927 Accumulated Data Length: 2039880
2019-07-31 16:31:05,052 Data arrived! len=2741
2019-07-31 16:31:05,053 Accumulated Data Length: 2042621
2019-07-31 16:31:06,793 Data arrived! len=7411
2019-07-31 16:31:06,794 Accumulated Data Length: 2050032
2019-07

2019-07-31 16:33:12,545 Accumulated Data Length: 2334596
2019-07-31 16:33:19,894 Data arrived! len=2457
2019-07-31 16:33:19,896 Accumulated Data Length: 2337053
2019-07-31 16:33:20,325 Data arrived! len=2756
2019-07-31 16:33:20,327 Accumulated Data Length: 2339809
2019-07-31 16:33:22,988 Data arrived! len=2672
2019-07-31 16:33:22,989 Accumulated Data Length: 2342481
2019-07-31 16:33:23,500 Data arrived! len=3034
2019-07-31 16:33:23,501 Accumulated Data Length: 2345515
2019-07-31 16:33:29,234 Data arrived! len=2479
2019-07-31 16:33:29,236 Accumulated Data Length: 2347994
2019-07-31 16:33:29,276 Data arrived! len=2768
2019-07-31 16:33:29,277 Accumulated Data Length: 2350762
2019-07-31 16:33:30,466 Data arrived! len=3162
2019-07-31 16:33:30,468 Accumulated Data Length: 2353924
2019-07-31 16:33:34,047 Data arrived! len=4085
2019-07-31 16:33:34,048 Accumulated Data Length: 2358009
2019-07-31 16:33:35,072 Data arrived! len=2906
2019-07-31 16:33:35,073 Accumulated Data Length: 2360915
2019-07

2019-07-31 16:35:59,256 Accumulated Data Length: 2635115
2019-07-31 16:36:00,075 Data arrived! len=2540
2019-07-31 16:36:00,076 Accumulated Data Length: 2637655
2019-07-31 16:36:05,706 Data arrived! len=4011
2019-07-31 16:36:05,707 Accumulated Data Length: 2641666
2019-07-31 16:36:06,014 Data arrived! len=3189
2019-07-31 16:36:06,015 Accumulated Data Length: 2644855
2019-07-31 16:36:11,753 Data arrived! len=3502
2019-07-31 16:36:11,755 Accumulated Data Length: 2648357
2019-07-31 16:36:13,183 Data arrived! len=2563
2019-07-31 16:36:13,184 Accumulated Data Length: 2650920
2019-07-31 16:36:15,127 Data arrived! len=3145
2019-07-31 16:36:15,128 Accumulated Data Length: 2654065
2019-07-31 16:36:17,892 Data arrived! len=2954
2019-07-31 16:36:17,893 Accumulated Data Length: 2657019
2019-07-31 16:36:19,429 Data arrived! len=4697
2019-07-31 16:36:19,430 Accumulated Data Length: 2661716
2019-07-31 16:36:19,431 Data arrived! len=2450
2019-07-31 16:36:19,432 Accumulated Data Length: 2664166
2019-07

ReadTimeoutError: HTTPSConnectionPool(host='stream.twitter.com', port=443): Read timed out.

# Program will keep running unless you interrupt

In [ ]:
from platform import python_version
print(python_version())

